In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Lendo arquivos

In [3]:
x_train = np.load('Xtreino5.npy')
y_train = np.load('ytreino5.npy')

x_test = np.load('Xteste5.npy')
y_test = np.load('yteste5.npy')


#função para reportar os resultados das combinaçẽos de parametros
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# 0.1 SVM Regressor
# 0.2 Medida de erro

In [12]:
list_exp_C = range(-5, 15, 5)
list_exp_gamma = range(-15, 3, 3)
list_epsilon = np.arange(0.05, 1, 0.5)


#Lista de parametros para utilizar nos demais
C_list = []
gamma_list = []

best_params = []
best_score = 0
mse = 0
for exp_gamma in list_exp_gamma:
    gamma = 2**exp_gamma
    gamma_list.append(gamma)
    for exp_C in list_exp_C:
        C = 2**exp_C
        C_list.append(C)
        for epilson in list_epsilon:
            clf = SVR(gamma=gamma, C=C, epsilon=epilson)
            clf.fit(x_train, y_train)  
            score = clf.score(x_test,y_test)        
            if score > best_score:
                best_score = score
                best_params = [gamma, C, epilson]
                mse = mean_squared_error(clf.predict(x_test), y_test)
print("Best Score: ",best_score ," with params: ", best_params," and MSE: ",mse)

C_list = np.unique(C_list)
gamma_list = np.unique(gamma_list)
list_epsilon = np.unique(list_epsilon)

Best Score:  0.7101346849881802  with params:  [0.000244140625, 1024, 0.05]  and MSE:  19.45487953760004


# 1 Random search

In [64]:
param_dist = {"C": [2**-5, 2**0, 2**5, 2**10, 2**15],
              "gamma": [2**-15, 2**-10, 2**-5, 2**0, 2**5],
              "epsilon": [0.05, 0.1, 0.5, 0.8, 1.0]}

clf = SVR()

random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=4)

random_search.fit(x_train, y_train)

report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.780 (std: 0.045)
Parameters: {'gamma': 3.0517578125e-05, 'epsilon': 0.5, 'C': 32768}

Model with rank: 2
Mean validation score: 0.725 (std: 0.042)
Parameters: {'gamma': 3.0517578125e-05, 'epsilon': 0.5, 'C': 1024}

Model with rank: 3
Mean validation score: 0.723 (std: 0.040)
Parameters: {'gamma': 3.0517578125e-05, 'epsilon': 1.0, 'C': 1024}



# 2 Grid seach

In [63]:
grid_search = GridSearchCV(clf, param_grid=param_dist, cv=5)

grid_search.fit(x_train, y_train)

report(grid_search.cv_results_)

/home/pazeto/anaconda3/envs/MO431A/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Model with rank: 1
Mean validation score: 0.746 (std: 0.040)
Parameters: {'C': 32768, 'epsilon': 1.0, 'gamma': 3.0517578125e-05}

Model with rank: 2
Mean validation score: 0.744 (std: 0.039)
Parameters: {'C': 32768, 'epsilon': 0.5, 'gamma': 3.0517578125e-05}

Model with rank: 3
Mean validation score: 0.744 (std: 0.034)
Parameters: {'C': 32768, 'epsilon': 0.1, 'gamma': 3.0517578125e-05}



# 3 Otimização bayesiana

In [4]:
# !pip install hyperopt
from hyperopt import hp, tpe, fmin, space_eval

def my_svr_fun(params):
    C, gamma, epsilon = params['C'], params['gamma'], params['epsilon']
    clf = SVR(gamma=gamma, C=C, epsilon=epsilon)
    clf.fit(x_train, y_train)  
    score = clf.score(x_test, y_test)  
    return -score

param_hyperopt= {
    'C': 2 ** hp.uniform('C', -5, 15),
    'gamma': 2 ** hp.uniform('gamma', -15, 3),
    'epsilon': hp.uniform('epsilon', 0.05, 1),
}

best_params = fmin(fn = my_svr_fun,
            space = param_hyperopt, algo=tpe.suggest, 
            max_evals = 125)

print("Melhores paramtros: ", space_eval(param_hyperopt, best_params))
print("Score: ", my_svr_fun(space_eval(param_hyperopt, best_params))*-1)

100%|██████████| 125/125 [00:50<00:00,  2.48it/s, best loss: -0.8252058541104953]
Melhores paramtros:  {'C': 18254.13344755772, 'epsilon': 0.15646708299651826, 'gamma': 3.194344320388373e-05}
Score:  0.8252058541104953


# 4 PSO

In [160]:
from pyswarm import pso

#primeiro elemento é o mais baixo e mais alte de C, segundo de gamma e terceiro de epilson
lb = [2**-5, 2**-15, 0.05]
ub = [2**15, 2**3, 1.0]

#dado que o pso vai esperar o menor valor e 
# queremos o melhor score devemos negativar o score, o mais baixo terá melhores params
def my_svr_fun(params):
#     print(params)
    C, gamma, epilson = params
    clf = SVR(gamma=gamma, C=C, epsilon=epilson)
    clf.fit(x_train, y_train)  
    score = clf.score(x_test,y_test)  
    return -score

best_params, best_score = pso(my_svr_fun, lb, ub, swarmsize=11, maxiter=11)
print("Parametros: ",best_params)
print("Score:", best_score*-1)

Stopping search: maximum iterations reached --> 11
Parametros:  [1.86712162e+04 3.05175781e-05 5.29600172e-02]
Score: 0.8296045658080052


# 5 Simulated annealing

In [ ]:
from simanneal import Annealer
import random

class SearchBestParamsSVR(Annealer):
    
    def move(self):
        
        """permuta a lista de parametros possíveis"""
        
        random.shuffle(self.state['C'])
        random.shuffle(self.state['gamma'])
        random.shuffle(self.state['epsilon'])
        
    def energy(self):
        """Calcula o melhor score"""
        
        C, gamma, epilson = self.state['C'][0], self.state['gamma'][0], self.state['epsilon'][0]
        clf = SVR(gamma=gamma, C=C, epsilon=epilson)
        clf.fit(x_train, y_train)  
        score = clf.score(x_test, y_test) 
        
        return -score
    
    
param_dist = {"C": [2**-5, 2**0, 2**5, 2**10, 2**15],
              "gamma": [2**-15, 2**-10, 2**-5, 2**0, 2**5],
              "epsilon": [0.05, 0.1, 0.5, 0.8, 1.0]}

tsp = SearchBestParamsSVR(param_dist)

params, score = tsp.anneal()

In [119]:
print("Parametros C gamma e epilson:",params['C'][0], params['gamma'][0], params['epsilon'][0],)
print("Score:", score*-1)

Pametros C gamma e epilson: 32768 3.0517578125e-05 0.05
Score: 0.8220660478366459


# 6 CMA-ES

In [73]:
# !pip install cma
import cma
# help(cma.fmin)
def my_svr_fun(params):
    C, gamma, epilson = params
    clf = SVR(gamma=gamma, C=C, epsilon=epilson)
    clf.fit(x_train, y_train)  
    score = clf.score(x_test,y_test)  
    return -score


lb = [2**-5, 2**-15, 0.05]
ub = [2**15, 2**3, 1]

opt = { 'bounds': [lb , ub], 'verb_disp':1000}
opt['scaling_of_variables'] = [2**10, 2**1, 0.01]

es = cma.fmin(my_svr_fun, [1, 1, 1], 2.5, opt)

print("Parametros C gamma e epilson: ", es[0])
print("Score: ", es[1]*-1)

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=769965, Wed May  1 11:49:26 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 -1.838206063591308e-04 1.0e+00 2.08e+00  2e+00  2e+00 0:00.1
    2     14 1.480056132296959e-04 1.4e+00 1.93e+00  1e+00  2e+00 0:00.2
    3     21 -9.133457200733330e-03 1.8e+00 1.83e+00  1e+00  2e+00 0:00.3
   22    154 -6.326321821002134e-01 1.2e+01 1.54e-01  1e-02  1e-01 0:03.5
   34    238 -6.475068819370783e-01 4.6e+01 7.24e-02  1e-03  6e-02 0:07.6
   46    322 -7.111023370763685e-01 3.3e+02 1.39e-01  1e-03  2e-01 0:13.1
   53    371 -7.681652760797661e-01 9.7e+02 7.36e-01  4e-03  2e+00 0:19.7
   56    392 -7.774315533913859e-01 1.1e+03 9.40e-01  3e-03  2e+00 0:27.1
   60    420 -7.824298371973549e-01 1.0e+03 8.63e-01  2e-03  1e+00 0:36.7
   64    448 -7.975142264359517e-01 1.2e+03 6.91e-01  1e-03  1e+00 0:48.6
   67    469 -7.968387811299247e-01 1.3e+03 8.05e-01  9e-04  2e+00 1:01.2
   70    490 -8.010360811